In [34]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import time
import glob
%matplotlib inline
from sklearn import preprocessing
import tables
import h5py
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [35]:
flights = pd.read_csv('BigData_Pruned_drop_Manu_V2.csv')
flights.head()

C:\Users\andre_000\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,MONTH,DOW,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE,ARRIVAL_DELAY,DIVERTED,CANCELLED,dept_temp,...,arr_precip,arr_alti,arr_vsby,arr_sky,CertificateDate,Manu,Model,DAYS,DEPARTURE_HOUR,ARRIVAL_HOUR
0,1,4,AS,ANC,SEA,1448,-22,0,0,33.80,...,0,30.32,10,CLR,2012.0,BOEING,737-990ER,0,9.1,13.5
1,1,4,DL,ANC,SEA,1448,-24,0,0,33.80,...,0,30.32,10,CLR,2001.0,AIRBUS,A321-211,0,9.8,14.2
2,1,4,AA,LAX,PBI,2330,-9,0,0,46.04,...,0,30.21,10,FEW,2009.0,BOEING,737-890,0,8.2,15.8
3,1,4,DL,DTW,PBI,1086,-8,0,0,17.96,...,0,30.21,10,FEW,2000.0,BOEING,737-832,0,13.0,16.1
4,1,4,AA,LGA,PBI,1035,-12,0,0,28.04,...,0,30.21,10,FEW,2014.0,AIRBUS,A320-232,0,12.5,15.5


In [39]:
toDrop = []
for i, k in enumerate(flights["CertificateDate"]):
    if (np.isnan(k)):
        toDrop.append(i)

In [40]:
len(toDrop)

14655

In [41]:
flights.drop(flights.index[toDrop], inplace=True)

In [42]:
len(flights)

4977281

In [43]:
flights.to_csv("BigData_Pruned_Drop.csv", index = False)

In [5]:
orig_airports = flights['ORIGIN_AIRPORT'].tolist()
dest_airports = flights['DESTINATION_AIRPORT'].tolist()

In [6]:
le_airports = preprocessing.LabelEncoder()
le_airports.fit(orig_airports)
#print(le_airports.classes_)
enc_orig_airports = le_airports.transform(orig_airports)
enc_dest_airports = le_airports.transform(dest_airports) 

enc_dest_airports

array([277, 277, 235, ..., 303, 203, 285], dtype=int64)

In [7]:
flights['ORIGIN_AIRPORT'] = enc_orig_airports
flights['DESTINATION_AIRPORT'] = enc_dest_airports
flights.head()

,MONTH,DOW,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE,ARRIVAL_DELAY,DIVERTED,CANCELLED,dept_temp,...,arr_precip,arr_alti,arr_vsby,arr_sky,CertificateDate,Manu,Model,DAYS,DEPARTURE_HOUR,ARRIVAL_HOUR
0,1,4,AS,17,277,1448,-22,0,0,33.80,...,0,30.32,10,CLR,2012.0,BOEING,737-990ER,0,9.1,13.5
1,1,4,DL,17,277,1448,-24,0,0,33.80,...,0,30.32,10,CLR,2001.0,AIRBUS,A321-211,0,9.8,14.2
2,1,4,AA,176,235,2330,-9,0,0,46.04,...,0,30.21,10,FEW,2009.0,BOEING,737-890,0,8.2,15.8
3,1,4,DL,93,235,1086,-8,0,0,17.96,...,0,30.21,10,FEW,2000.0,BOEING,737-832,0,13.0,16.1
4,1,4,AA,182,235,1035,-12,0,0,28.04,...,0,30.21,10,FEW,2014.0,AIRBUS,A320-232,0,12.5,15.5


In [8]:
manufacts = flights['Manu'].tolist()

le_manu = preprocessing.LabelEncoder()
le_manu.fit(manufacts)
#print(le_airports.classes_)
enc_manufacts = le_manu.transform(manufacts)

flights['Manu'] = enc_manufacts
flights.head()

,MONTH,DOW,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE,ARRIVAL_DELAY,DIVERTED,CANCELLED,dept_temp,...,arr_precip,arr_alti,arr_vsby,arr_sky,CertificateDate,Manu,Model,DAYS,DEPARTURE_HOUR,ARRIVAL_HOUR
0,1,4,AS,17,277,1448,-22,0,0,33.80,...,0,30.32,10,CLR,2012.0,6,737-990ER,0,9.1,13.5
1,1,4,DL,17,277,1448,-24,0,0,33.80,...,0,30.32,10,CLR,2001.0,1,A321-211,0,9.8,14.2
2,1,4,AA,176,235,2330,-9,0,0,46.04,...,0,30.21,10,FEW,2009.0,6,737-890,0,8.2,15.8
3,1,4,DL,93,235,1086,-8,0,0,17.96,...,0,30.21,10,FEW,2000.0,6,737-832,0,13.0,16.1
4,1,4,AA,182,235,1035,-12,0,0,28.04,...,0,30.21,10,FEW,2014.0,1,A320-232,0,12.5,15.5


In [9]:
models = flights['Model'].tolist()

le_model = preprocessing.LabelEncoder()
le_model.fit(models)
#print(le_airports.classes_)
enc_models = le_model.transform(models)

flights['Model'] = enc_models
flights.head()

,MONTH,DOW,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE,ARRIVAL_DELAY,DIVERTED,CANCELLED,dept_temp,...,arr_precip,arr_alti,arr_vsby,arr_sky,CertificateDate,Manu,Model,DAYS,DEPARTURE_HOUR,ARRIVAL_HOUR
0,1,4,AS,17,277,1448,-22,0,0,33.80,...,0,30.32,10,CLR,2012.0,6,53,0,9.1,13.5
1,1,4,DL,17,277,1448,-24,0,0,33.80,...,0,30.32,10,CLR,2001.0,1,103,0,9.8,14.2
2,1,4,AA,176,235,2330,-9,0,0,46.04,...,0,30.21,10,FEW,2009.0,6,46,0,8.2,15.8
3,1,4,DL,93,235,1086,-8,0,0,17.96,...,0,30.21,10,FEW,2000.0,6,45,0,13.0,16.1
4,1,4,AA,182,235,1035,-12,0,0,28.04,...,0,30.21,10,FEW,2014.0,1,102,0,12.5,15.5


In [10]:
arr_sky = flights['arr_sky'].tolist()
dept_sky = flights['dept_sky'].tolist()

le_sky = preprocessing.LabelEncoder()
le_sky.fit(arr_sky)
#print(le_airports.classes_)
enc_arr_sky = le_sky.transform(arr_sky)
enc_dept_sky = le_sky.transform(dept_sky)

flights['dept_sky'] = enc_dept_sky
flights['arr_sky'] = enc_arr_sky
flights.head()

,MONTH,DOW,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE,ARRIVAL_DELAY,DIVERTED,CANCELLED,dept_temp,...,arr_precip,arr_alti,arr_vsby,arr_sky,CertificateDate,Manu,Model,DAYS,DEPARTURE_HOUR,ARRIVAL_HOUR
0,1,4,AS,17,277,1448,-22,0,0,33.80,...,0,30.32,10,1,2012.0,6,53,0,9.1,13.5
1,1,4,DL,17,277,1448,-24,0,0,33.80,...,0,30.32,10,1,2001.0,1,103,0,9.8,14.2
2,1,4,AA,176,235,2330,-9,0,0,46.04,...,0,30.21,10,2,2009.0,6,46,0,8.2,15.8
3,1,4,DL,93,235,1086,-8,0,0,17.96,...,0,30.21,10,2,2000.0,6,45,0,13.0,16.1
4,1,4,AA,182,235,1035,-12,0,0,28.04,...,0,30.21,10,2,2014.0,1,102,0,12.5,15.5


In [11]:
airlines = flights['AIRLINE'].tolist()
le_airline = preprocessing.LabelEncoder()
le_airline.fit(airlines)
#print(le_airports.classes_)
enc_airlines = le_airline.transform(airlines)

flights['AIRLINE'] = enc_airlines
flights.head()

,MONTH,DOW,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE,ARRIVAL_DELAY,DIVERTED,CANCELLED,dept_temp,...,arr_precip,arr_alti,arr_vsby,arr_sky,CertificateDate,Manu,Model,DAYS,DEPARTURE_HOUR,ARRIVAL_HOUR
0,1,4,1,17,277,1448,-22,0,0,33.80,...,0,30.32,10,1,2012.0,6,53,0,9.1,13.5
1,1,4,3,17,277,1448,-24,0,0,33.80,...,0,30.32,10,1,2001.0,1,103,0,9.8,14.2
2,1,4,0,176,235,2330,-9,0,0,46.04,...,0,30.21,10,2,2009.0,6,46,0,8.2,15.8
3,1,4,3,93,235,1086,-8,0,0,17.96,...,0,30.21,10,2,2000.0,6,45,0,13.0,16.1
4,1,4,0,182,235,1035,-12,0,0,28.04,...,0,30.21,10,2,2014.0,1,102,0,12.5,15.5


In [14]:
list(flights)

['MONTH',
 'DOW',
 'AIRLINE',
 'ORIGIN_AIRPORT',
 'DESTINATION_AIRPORT',
 'DISTANCE',
 'ARRIVAL_DELAY',
 'DIVERTED',
 'CANCELLED',
 'dept_temp',
 'dept_dew',
 'dept_hum',
 'dept_windspd',
 'dept_precip',
 'dept_alti',
 'dept_vsby',
 'dept_sky',
 'arr_temp',
 'arr_dew',
 'arr_hum',
 'arr_windspd',
 'arr_precip',
 'arr_alti',
 'arr_vsby',
 'arr_sky',
 'CertificateDate',
 'Manu',
 'Model',
 'DAYS',
 'DEPARTURE_HOUR',
 'ARRIVAL_HOUR']

In [15]:
features = ['MONTH',
 'DOW',
 'AIRLINE',
 'ORIGIN_AIRPORT',
 'DESTINATION_AIRPORT',
 'DISTANCE',
 'DIVERTED',
 'CANCELLED',
 'dept_temp',
 'dept_dew',
 'dept_hum',
 'dept_windspd',
 'dept_precip',
 'dept_alti',
 'dept_vsby',
 'dept_sky',
 'arr_temp',
 'arr_dew',
 'arr_hum',
 'arr_windspd',
 'arr_precip',
 'arr_alti',
 'arr_vsby',
 'arr_sky',
 'CertificateDate',
 'Manu',
 'Model',
 'DAYS',
 'DEPARTURE_HOUR',
 'ARRIVAL_HOUR']

In [32]:
toDrop = []
for i, k in enumerate(flights["CertificateDate"]):
    if (np.isnan(k)):
        toDrop.append(k)

4977281


MONTH                    12.00
DOW                       3.00
AIRLINE                   8.00
ORIGIN_AIRPORT          114.00
DESTINATION_AIRPORT     174.00
DISTANCE               2173.00
ARRIVAL_DELAY             8.00
DIVERTED                  0.00
CANCELLED                 0.00
dept_temp                78.98
dept_dew                 75.02
dept_hum                 87.70
dept_windspd              8.00
dept_precip               0.00
dept_alti                30.18
dept_vsby                10.00
dept_sky                  2.00
arr_temp                 41.00
arr_dew                  15.98
arr_hum                  35.85
arr_windspd               5.00
arr_precip                0.00
arr_alti                 30.14
arr_vsby                 10.00
arr_sky                   1.00
CertificateDate            NaN
Manu                     28.00
Model                   145.00
DAYS                    364.00
DEPARTURE_HOUR            1.10
ARRIVAL_HOUR              3.50
Name: 4977281, dtype: float64

In [17]:
X = flights[features]
y = flights["ARRIVAL_DELAY"]

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=0)

In [29]:
model = LogisticRegression()
model = model.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').